In [54]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file (pd.read_csv)
#Input data files are available in the "../input/" directory.
#For example, running this will list all files under the input dir.

import os
for dirname, _, filenames in os.walk("data"):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        

data/test.csv
data/train.csv
data/gender_submission.csv


In [47]:
data = pd.read_csv("data/train.csv")
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [55]:
test_data = pd.read_csv("data/test.csv")
test_data.head()


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [11]:
women = data.loc[data.Sex == "female"]["Survived"]
rate_women = sum(women)/len(women)
print("% of women who survived:", rate_women)

% of women who survived: 0.7420382165605095


In [7]:
men = data.loc[data.Sex == "male"]["Survived"]
rate_men = sum(men)/len(men)
print("% of men who survived:", rate_men)

% of men who survived: 0.18890814558058924


In [49]:
#Data preprocessing:
#makes male and female

from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder()
sex_encoded = encoder.fit_transform(data[["Sex"]]).toarray()
data = data.drop("Sex", axis = 1)
data = pd.concat([data, pd.DataFrame(sex_encoded, columns=["female", "male"])], axis=1)
print(data.head())


   PassengerId  Survived  Pclass   
0            1         0       3  \
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name   Age  SibSp  Parch   
0                            Braund, Mr. Owen Harris  22.0      1      0  \
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  38.0      1      0   
2                             Heikkinen, Miss. Laina  26.0      0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  35.0      1      0   
4                           Allen, Mr. William Henry  35.0      0      0   

             Ticket     Fare Cabin Embarked  female  male  
0         A/5 21171   7.2500   NaN        S     0.0   1.0  
1          PC 17599  71.2833   C85        C     1.0   0.0  
2  STON/O2. 3101282   7.9250   NaN        S     1.0   0.0  
3            113803  53.1000  C123        S     1.0   0.0  
4            373450   8.05

In [60]:
#Data preprocessing for sex 0/1

from sklearn.preprocessing import LabelEncoder

#Create a LabelEncoder object
le = LabelEncoder()

le.fit(data["Sex"])
df["sex_encoded"] = le.transform(df['Sex'])

print(data["Sex"].unique())

KeyError: 'Sex'

In [58]:
#Delete NaNs:

data = data.fillna(0)

In [27]:
data.tail()

,PassengerId,Survived,Pclass,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,female,male
886,887,0,2,"Montvila, Rev. Juozas",27.0,0,0,211536,13.00,0,S,0.0,1.0
887,888,1,1,"Graham, Miss. Margaret Edith",19.0,0,0,112053,30.00,B42,S,1.0,0.0
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",0.0,1,2,W./C. 6607,23.45,0,S,1.0,0.0
889,890,1,1,"Behr, Mr. Karl Howell",26.0,0,0,111369,30.00,C148,C,0.0,1.0
890,891,0,3,"Dooley, Mr. Patrick",32.0,0,0,370376,7.75,0,Q,0.0,1.0


In [56]:
### Training data using decision tree

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

features = ["Pclass", "female", "male", "Age", "SibSp", "Parch", "Fare"]

# here'll be some data preprocessing but not now.

train_data, val_data, train_labels, val_labels =  train_test_split(data[features], data["Survived"], test_size = 0.2, random_state = 42)


In [59]:
#Train a decision tree classifier

model = DecisionTreeClassifier(max_depth = 3, random_state = 42)
model.fit(train_data, train_labels)

#Make predictions on validation set
pred_labels = model.predict(val_data)
accuracy = accuracy_score(val_labels, pred_labels)
print("Accuracy:", accuracy)
print(val_labels)

ValueError: Input X contains NaN.
DecisionTreeClassifier does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [42]:
#Checking
feature_names = model.feature_names_in_
print(feature_names)

['Pclass' 'female' 'male' 'Age' 'SibSp' 'Parch' 'Fare']


In [65]:
#Sumbit predictions for Kaggle:
test_passenger_ids = test_data["PassengerId"]
test_data["female"] = 0
test_data["male"] = 0
test_data = test_data.fillna(0)

for i in test_data["Sex"]:
    if i == "female":
        test_data.at[i, "female"] = 1
        test_data.at[i, "male"] = 0
    elif i == "male":
        test_data.at[i, "female"] = 0
        test_data.at[i, "male"] = 1
    else:
        test_data.at[i, "female"] = 0
        test_data.at[i, "male"] = 0



test_data = test_data[feature_names]

test_predictions = model.predict(test_data)
submission_dtc = pd.DataFrame({"PassengerId": test_passenger_ids, "Survived": test_predictions})

submission_dtc_to_csv("submission.csv", index=False)

KeyError: 'PassengerId'

In [64]:
test_data = test_data.fillna(0)
print(test_data.isnull().sum())

Pclass    0
female    0
male      0
Age       0
SibSp     0
Parch     0
Fare      0
dtype: int64
